Data Source: https://designcensus.org/

In [30]:
import numpy as np
import pandas as pd
import math
from IPython.display import display

In [31]:
df = pd.read_csv('DesignCensus2017_Data.csv')

In [32]:
print('There a a total of ' + str(len(df.columns)) + ' columns.')

There a a total of 43 columns.


### Column dictionary

In [33]:
col_dict = {}
for col in df.columns:
    index, desc = col.split(' --')
    col_dict['V_' + str(index)] = desc

In [34]:
col_dict

{'V_1': " I've worked in design for:",
 'V_10': 'My immediate team is:',
 'V_11': 'My work touches these industries or sectors:',
 'V_12': 'I’ve been at my current place of work for:',
 'V_13': 'My role is:',
 'V_14': 'My role is:',
 'V_15': 'My role is:',
 'V_16': "I've been in my current role for:",
 'V_17': 'I typically work:',
 'V_18': 'My annual salary is:',
 'V_19': 'My job provides:',
 'V_2': 'I primarily work in:',
 'V_20': 'In my current role, there is:',
 'V_21': 'In my current role, I am:',
 'V_22': 'My job is:',
 'V_23': 'My job stability is:',
 'V_24': 'In addition to my job, I also:',
 'V_25': 'I fuel my daily creativity with coffee:',
 'V_26': "Highest level of education I've completed:",
 'V_27': 'I learned design:',
 'V_28': 'My major focus of study was:',
 'V_29': "Over my career, I've had:",
 'V_3': "I'm also good at:",
 'V_30': "I've shifted my specialty or focus area:",
 'V_31': 'I stay current with design by:',
 'V_32': "I'm a member of:",
 'V_33': "I'd like to ge

### Classify Columns

In [35]:
df.columns = map(lambda x : 'V_' + str(x+1), list(range(len(df.columns)))) 

In [36]:
continous_cols = ['V_18', 'V_37']

In [37]:
pipe_col = ['V_2', 'V_3','V_5','V_7','V_11','V_19',\
            'V_24', 'V_27', 'V_28', 'V_31', 'V_32',\
            'V_33', 'V_40', 'V_41', 'V_42', 'V_43']

In [38]:
cate_cols = list(df.columns)
for col in continous_cols:
    cate_cols.remove(col)

### Summary of Null Value

In [39]:
dff = pd.DataFrame(columns=['Column','Desc', 'Null'])
for col in df.columns:
    num_null = df[col].isnull().sum()
    dff = dff.append({'Column': col, 'Desc' : col_dict[col], 'Null' : num_null }, ignore_index=True)
dff

,Column,Desc,Null
0,V_1,I've worked in design for:,0
1,V_2,I primarily work in:,0
2,V_3,I'm also good at:,0
3,V_4,I am currently:,0
4,V_5,My company specializes in:,7198
5,V_6,My company is:,7317
6,V_7,I teach:,12800
7,V_8,I'm not working because:,12831
8,V_9,My organization has:,409
9,V_10,My immediate team is:,402


### Deal with columns with pipe

In [41]:
df['V_36'] = df['V_36'].astype(np.object)
df = df.replace(np.nan,'', regex=True)
pipe_col_dict = {}

for col in pipe_col:
    options = set()
    for row in df[col]:
        arrs = row.split('|') 
        for arr in arrs:
            if arr != '':
                options.add(arr)
    
    pipe_col_dict[col] = {}
    
    index = 1
    for option in options:
        sub_col = col + '_' + str(index)
        df.loc[:,sub_col] = 0
        pipe_col_dict[col][option] = sub_col
        index += 1

In [ ]:
for col in pipe_col:
    row_num = 0;
    for row in df[col]:
        arrs = row.split('|') 
        for arr in arrs:
            if arr != '':
                sub_col = pipe_col_dict[col][arr]
                df.loc[row_num, sub_col] += 1
        row_num += 1

In [ ]:
df.to_csv('output.csv')

### Exclude null, outliers, etc

TBD

### End of preprocessing

In [42]:
df = pd.read_csv('output.csv')
num_records = len(df)

In [43]:
def summary_cate_col(col):
    if col in continous_cols:
        return
    
    elif col in pipe_col:
        dff = pd.DataFrame(columns=['Options','Count','Percent'])
        for key, value in pipe_col_dict[col].items():
            count = df[value].sum()
            dff = dff.append({'Options': key, 'Count' : count}, ignore_index=True)
    
    else:
        dff = pd.DataFrame(df[col].value_counts())
        dff.reset_index(inplace=True)
        dff.columns = ['Options','Count']
    
    dff['Percent'] = dff.Count / num_records
    dff = dff.style.format({'Percent': '{:,.2%}'.format})
    return dff

### Basic Summary of each column (only show top 10)

In [48]:
for col in cate_cols:
    display(str(col) + ' : ' + col_dict[col])
    display(summary_cate_col(col).sort_values(by = 'Percent', ascending = False).head(10))

"V_1 :  I've worked in design for:"

AttributeError: 'Styler' object has no attribute 'sort_values'

In [51]:
a = summary_cate_col(col)

In [52]:
a.ssort_values(by = 'Percent')

AttributeError: 'Styler' object has no attribute 'ssort_values'

In [53]:
a.dtype

AttributeError: 'Styler' object has no attribute 'dtype'